In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bybit import *
from passivbot import calc_diff
import matplotlib.pyplot as plt
from pprint import PrettyPrinter

In [3]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [4]:
exchange = 'bybit'
user = 'e'
settings = load_settings(exchange, user)
settings['exchange'] = exchange
s = settings['symbol']

2021-02-19T22:52:48  settings for user e not found, using default settings 

loaded settings:
{'balance_pct': 0.528,
 'config_name': 'BTCUSD_triple_down_52_days_2021-02-14',
 'cross_mode': True,
 'ddown_factor': 2.186,
 'entry_qty_pct': 0.02636,
 'grid_coefficient': 639.4,
 'grid_spacing': 0.00221,
 'indicator_settings': {'do_long': True,
                        'do_shrt': True,
                        'funding_fee_collect_mode': False,
                        'ohlcv_rsi': {'n_periods': 21, 'period_ms': 1800000},
                        'tick_ema': {'span': 6355.0, 'spread': 0.0}},
 'leverage': 59.0,
 'logging_level': 0,
 'market_stop_loss': False,
 'max_markup': 0.02168,
 'min_close_qty_multiplier': 0.6,
 'min_markup': 0.00047,
 'n_close_orders': 17.0,
 'n_entry_orders': 8,
 'stop_loss_liq_diff': 0.1149,
 'stop_loss_pos_price_diff': 0.0947,
 'stop_loss_pos_reduction': 0.012,
 'symbol': 'BTCUSD'}


In [5]:
try:
    await bot.cc.close()
except:
    pass
bot = await create_bot(user, settings)
cc = bot.cc

In [9]:
await bot.update_position()
bot.position, bot.position['price'] * (1 + bot.stop_loss_pos_price_diff)

({'size': -149,
  'price': 52949.91435618,
  'leverage': 100.0,
  'liquidation_price': 87396.0,
  'equity': 0.00098984,
  'wallet_balance': 0.00112444,
  'cost': 0.0028139799999999356,
  'margin_cost': 4.769457627118535e-05,
  'predicted_funding_rate': 0.0001},
 57964.271245710246)

In [8]:
bot.calc_orders()

[{'side': 'sell',
  'qty': 325.0,
  'price': 59088.5,
  'type': 'limit',
  'reduce_only': False,
  'custom_id': 'entry'},
 {'side': 'sell',
  'qty': 1036.0,
  'price': 74145.5,
  'type': 'limit',
  'reduce_only': False,
  'custom_id': 'entry'},
 {'side': 'buy',
  'qty': 149.0,
  'price': 52925.0,
  'type': 'limit',
  'reduce_only': True,
  'custom_id': 'close'}]

In [21]:
56943 / 51038

1.1156981072926055

In [ ]:
bot.flush_stuck_locks()

In [ ]:
ccc = await bot.create_orders([{'side': 'buy', 'qty': 1.0, 'price': 42000, 'type': 'limit', 'reduce_only': False, 'custom_id': 'test'},
                               {'side': 'sell', 'qty': 1.0, 'price': 51000, 'type': 'limit', 'reduce_only': False, 'custom_id': 'test'}])

In [ ]:
ccc

In [ ]:
await bot.update_open_orders()
bot.open_orders.append({'order_id': 'hello'})
await bot.cancel_orders(bot.open_orders[-3:])

In [ ]:
0.2956 / 28

In [ ]:
ts_to_date((millis_now - millis_now % (1000 * 60 * 60 * 7)) / 1000)

In [ ]:
millis_now % (1000 * 60 * 60 * 8)

In [ ]:
xs = [millis_now - z * 13 * 60 * 1000 for z in range(200)]
ys = [x - x % (1000 * 60 * 60 * 8) - 60 * 60 * 1000 for x in xs]
[(ts_to_date(x / 1000), ts_to_date(y / 1000)) for x, y in zip(xs, ys)]


In [ ]:
await bot.init_my_trades()

In [ ]:
len(bot.my_trades)

In [ ]:
mtdf = pd.DataFrame(bot.my_trades)
le = mtdf[(mtdf.side == 'buy') & (mtdf.custom_id.str.startswith('entry'))]
se = mtdf[(mtdf.side == 'sell') & (mtdf.custom_id.str.startswith('entry'))]
lx = mtdf[(mtdf.side == 'sell') & (mtdf.custom_id.str.startswith('close'))]
sx = mtdf[(mtdf.side == 'buy') & (mtdf.custom_id.str.startswith('close'))]

le.price.plot(style='b.-')
lx.price.plot(style='ro')

In [ ]:
se.price.plot(style='r.-')
sx.price.plot(style='bo')

In [ ]:
mtdf.tail(20)

In [ ]:
bot.calc_positions()

In [ ]:
await asyncio.gather(*[bot.update_position(), bot.update_my_trades(), bot.init_order_book()])
positions = bot.calc_positions()
bot.position, positions

In [ ]:
bot.calc_orders_simultaneous_mode()

In [ ]:
xs = await cc.v2_private_get_order_list(params={'symbol': 'BTCUSD', 'limit': 50})
ys = await cc.v2_private_get_execution_list(params={'symbol': 'BTCUSD', 'limit': 50, 'order': 'desc'})

In [ ]:
all_orders = []
for e in xs['result']['data']:
    if e['order_status'] == 'Filled':
        all_orders.append(e)
all_orders = sorted(all_orders, key=lambda q: q['created_at'], reverse=True)
executed_orders = []
for e in ys['result']['trade_list']:
    executed_orders.append({**e, **{'datetime': ts_to_date(e['trade_time_ms'] / 1000)}})
executed_orders = sorted(executed_orders, key=lambda q: q['trade_time_ms'], reverse=True)

In [ ]:
pd.DataFrame(all_orders)

In [ ]:
pd.DataFrame(executed_orders)

In [ ]:
for ao, eo in zip(all_orders[:4], executed_orders[:4]):
    pp.pprint([ao, eo])

In [ ]:
all_orders[-2:]

In [ ]:
executed_orders[-2:]

In [ ]:
import logging

In [ ]:
logging.basicConfig(filename='logs/example_log.log', level=logging.DEBUG)

In [ ]:
logging.error('anothertest whatsmore error')

In [ ]:
with open('logs/example_log.log') as f:
    print(f.read())

In [ ]:
mt[-10:]

In [ ]:
long_qty = 0.0
long_cost = 0.0
shrt_qty = 0.0
shrt_cost = 0.0
w = ''
for t in mt:
    cost = t['qty'] / t['price']
    if t['side'] == 'buy':
        if t['custom_id'].startswith('entry'):
            long_qty += t['qty']
            long_cost += cost
            w = 'long entry'
        elif t['custom_id'].startswith('close'):
            shrt_qty -= t['qty']
            shrt_cost -= cost
            if shrt_qty <= 0.0 or shrt_cost <= 0.0:
                shrt_qty = 0.0
                shrt_cost = 0.0
            w = 'shrt close'
    elif t['side'] == 'sell':
        if t['custom_id'].startswith('entry'):
            shrt_qty += t['qty']
            shrt_cost += cost
            w = 'shrt entry'
        elif t['custom_id'].startswith('close'):
            long_qty -= t['qty']
            long_cost -= cost
            if long_qty <= 0.0 or long_cost <= 0.0:
                long_qty = 0.0
                long_cost = 0.0
            w = 'long close'
    long_vwap = long_qty / long_cost if long_cost else 0.0
    shrt_vwap = shrt_qty / shrt_cost if shrt_cost else 0.0
    print(f"{w} {t['qty']} {t['price']} long {long_qty:.2f} @ {long_vwap:.1f} shrt {shrt_qty:.2f} @ {shrt_vwap:.1f}")


In [ ]:
long_qty, long_cost, shrt_qty, shrt_cost

In [ ]:
trades = await asyncio.gather(*
    [cc.v2_private_get_execution_list(params={'symbol': 'BTCUSD', 'limit': 200, 'order': 'desc', 'page': page})
     for page in range(1, 3)]
)
mt = sorted(flatten([t_['result']['trade_list'] for t_ in trades]), key=lambda x: x['trade_time_ms'])
mt = [e for e in mt if e['trade_time_ms'] == e['trade_time_ms']]

In [ ]:
mtdf = pd.DataFrame(mt).dropna().sort_values('trade_time_ms')
mtdf = mtdf[~mtdf.order_id.duplicated()]
mtdf.index = np.arange(len(mtdf))
mtdf = mtdf.join(pd.Series((mtdf.trade_time_ms / 1000).apply(ts_to_date), name='datetime'))
mtdf

In [ ]:
mt[0]

In [ ]:
pos_size = 0.0
buy_vwap = 0.0
sel_vwap = 0.0
for t in mt:
    if t['side'] == 'Buy':
        if t['order_link_id'].startswith('entry'):
            print('long entry', t['exec_price'])
        elif t['order_link_id'].startswith('close'):
            print('shrt close', t['exec_price'])
    elif t['side'] == 'Sell':
        if t['order_link_id'].startswith('entry'):
            print('shrt entry', t['exec_price'])
        elif t['order_link_id'].startswith('close'):
            print('long close', t['exec_price'])


In [ ]:
mdf = pd.DataFrame(mt).drop(['user_id', 'exec_id', 'exec_time', 'cross_seq'], axis=1).sort_values('trade_time_ms').dropna()
mdf = mdf[~mdf.order_id.duplicated()]
mtdf = mdf[mdf.trade_time_ms == mdf.trade_time_ms]
mtdf = mtdf.reindex(np.arange(len(mtdf)))
#mtdf.loc[:,'datetime'] = ((mtdf.trade_time_ms / 1000).apply(ts_to_date).values)

In [ ]:
(mtdf.trade_time_ms / 1000).apply(ts_to_date)

In [ ]:
mdfzz = mdf[mdf.trade_time_ms == mdf.trade_time_ms]
(mdfzz.trade_time_ms / 1000).apply(ts_to_date)

In [ ]:
mtdf = pd.DataFrame(mt['result']['trade_list']).drop(['user_id', 'exec_id', 'exec_time',
                                                      'cross_seq'], axis=1)

mtdf.loc[:,'datetime'] = ((mtdf.trade_time_ms / 1000).apply(ts_to_date).values)
mtdf

In [ ]:
'/XMRUSDT/xmr_with_soft_stop/2021-01-29T15_37_21/'[:-20]

In [ ]:
mt

In [ ]:
mt = await cc.fetch_my_trades('BTC/USD', since=start_time)
mtdf = pd.DataFrame(mt).drop(['info'], axis=1)
mtdf

In [ ]:
calc_isolated_long_liq_price(0.0, 0.0, 30000, leverage=2)

In [ ]:
calc_isolated_shrt_liq_price(0.0, 0.0, 30000, leverage=10)

In [ ]:
ts_to_date(cc.milliseconds() / 1000)[:10]

In [ ]:
await cc.v2_private_post

In [ ]:
await bot.init_order_book()
await bot.update_position()
pprint.pprint(bot.position)
print(bot.ob)
#bot.position['liquidation_price'] = 31200
#bot.position['price'] = 38600
#bot.position['size'] = 650
#bot.price = 38500
orders = bot.calc_orders()
print('liq diff', calc_diff(bot.position['liquidation_price'], bot.price), bot.price)
orders = [{**o, **{'diff': abs(o['price'] - bot.price) / bot.price}} for o in orders]
sorted(orders, key=lambda x: x['price'], reverse=True)

In [ ]:
await bot.execute_order({'side': 'sell',
  'qty': 1.0,
  'price': 35837.5,
  'type': 'limit',
  'reduce_only': False,
  'diff': 0.021331471401293853})

In [ ]:
await cc.private_get_wallet_balance()

In [ ]:
calc_default_qty(1.0, 1.0, 0.002 * 34000, 0.03)

In [ ]:
0.002 * 34000

In [ ]:
bot.dynamic_grid = False
await bot.init_order_book()
await bot.update_position()
print(bot.position, bot.ob)
bot.position['liquidation_price'] = 36500
#bot.position['price'] = 38600
#bot.position['size'] = 650
#bot.price = 38500
orders = bot.calc_orders()
print('liq diff', calc_diff(bot.position['liquidation_price'], bot.price), bot.price)
orders = [{**o, **{'diff': abs(o['price'] - bot.price) / bot.price}} for o in orders]
sorted(orders, key=lambda x: x['price'], reverse=True)

In [ ]:
np.diff(sorted([o['price'] for o in orders if o['side'] == 'sell'], reverse=True))

In [ ]:
%timeit calc_diff(3,4)

In [ ]:
%timeit calc_diff(np.random.random(5000) - 0.5, np.random.random(5000) - 0.5)

In [ ]:
%timeit calc_diff(np.random.random(5000) - 0.5, np.random.random(5000) - 0.5)

In [ ]:
np.random.random(5) - 0.5

In [ ]:
from passivbot import round_
eq = 0.00167
d = 100
round_dn(eq, round_up(eq / d, 0.0001))

In [ ]:
bot.position

In [ ]:
await bot.init_order_book()
await bot.update_position()
created = await bot.cancel_and_create()

In [ ]:
sorted([(e['price'], e['qty'], e['side']) for e in bot.open_orders])

In [ ]:
orders = bot.calc_orders()
orders

In [ ]:
round_up(bot.price * bot.grid_spacing / 4, bot.price_step)

In [ ]:
17032 // 1000

In [ ]:
await bot.update_position()
bot.position, bot.open_orders

In [ ]:
await bot.cancel_and_create()

In [ ]:
await bot.start_websocket()

In [ ]:
await bot.update_position()
bot.position

In [ ]:
await bot.update_position()
bot.position

In [ ]:
price = 34328.87058016
ask_price = 34340.5
(price + ask_price) / 2

In [ ]:
(34245.5 + 34075.0) / 2

In [ ]:
34245.5 * 0.0000292

In [ ]:
34075.0 * 0.00002934

In [ ]:
'market short, pos_price, trade_price', 34164.67, 34162.5

In [ ]:
'limit long, pos_price, trade_price', 34094.78, 34090.5

In [ ]:
'limit long reentry pos_price, trade prices', 34013.6, [34090.5, 33924.0]

In [ ]:
pos_price = 34013.6
entry_prices = [34090.5, 33924.0]
avg = sum(entry_prices) / len(entry_prices)
pos_price, avg, pos_price / avg

In [ ]:
34090.5 * (1 + 0.00025 / 2)

In [ ]:
avg * (1 + 0.00025 / 2)

In [ ]:
bot.calc_orders()

In [ ]:
etht = await cc.public_get_trading_records(params={'symbol': 'ETHUSD'})

In [ ]:
await fetch_trades(cc, 'ETHUSD')

In [ ]:
[e['time'] for e in etht['result']]

In [ ]:
mt = await cc.fetch_my_trades('BTC/USD', limit=200, since=0)
for q in range(14):
    print(q)
    new_mt = await cc.fetch_my_trades('BTC/USD', limit=200, since=mt[-1]['timestamp'])
    if new_mt[-1]['id'] == mt[-1]['id']:
        break
    mt += new_mt


In [ ]:
mtdf = pd.DataFrame(mt).sort_values('timestamp')
mtdf = mtdf[~mtdf['id'].duplicated()]
mtdf.index = range(len(mtdf))

In [ ]:
mtdf

In [ ]:
mtdf = mtdf.drop(['info', 'id', 'order'], axis=1)
fee_cost_d = dict(mtdf.fee)
fee_cost = pd.Series([fee_cost_d[k]['cost'] for k in sorted(fee_cost_d)], name='fee_cost')
fee_currency = pd.Series([fee_cost_d[k]['currency'] for k in sorted(fee_cost_d)], name='fee_currency')
fee_rate = pd.Series([fee_cost_d[k]['rate'] for k in sorted(fee_cost_d)], name='fee_rate')
mtdf = mtdf.join(pd.concat([fee_cost, fee_currency, fee_rate], axis=1)).drop('fee', axis=1)

In [ ]:
mtdf

In [ ]:
bdf = mtdf[mtdf.side == 'buy']
sdf = mtdf[mtdf.side == 'sell']
bvwap = bdf.amount.sum() / bdf.cost.sum()
svwap = sdf.amount.sum() / sdf.cost.sum()
(svwap / bvwap - 1) * (bdf.cost.sum() + sdf.cost.sum()) / 2 - mtdf.fee_cost.sum()

In [ ]:
mtdf.fee_cost.sum()

In [ ]:
mtdf.head(40)

In [ ]:
n_days = (time() * 1000 - 1608681600000) / 1000 / 60 / 60 / 24
net_pnl = 0.00110581
eq = 0.004933
gain = (net_pnl + eq) / eq
gain ** (1 / n_days)

In [ ]:
100 / 26800 / 91 * 2**11

In [ ]:
0.005 * 1.04**7

In [ ]:
await cc.private_get_trade_closed_pnl_list(params={'symbol': s, 'start_time': 0})

In [ ]:
100 * 49

In [ ]:
await bot.fetch_trades()

In [ ]:
await bot.init_emas()

In [ ]:
bot.emas

In [ ]:
await bot.cc.private_get_order(params={'symbol': bot.symbol})

In [ ]:
await bot.create_orders([{'side': 'buy', 'amount': 1.0, 'price': 21000}])
bot.position, bot.open_orders

In [ ]:
0.001 * 2**10 * 25000 / 100 * 6

In [ ]:
bot.position['size'] = -1.024

In [ ]:
bot.calc_orders()

In [ ]:
await bot.update_position()
bot.position

In [ ]:
await bot.update_open_orders()
bot.position

In [ ]:
await bot.create_orders()

In [ ]:
bot.open_orders

In [ ]:
await bot.start_websocket()

In [ ]:
rpath = 'historical_data/agg_trades_futures/'
npath = 'historical_data/binance/agg_trades_futures/'
dirs = [f for f in os.listdir(rpath) if 'USD' in f]
for f in dirs:
    csvs = [x for x in os.listdir(rpath + f) if '.csv' in x]
    for e in csvs:
        fullpath = rpath + f + '/' + e
        with open(fullpath) as y:
            lines = y.read()
        updated = lines.replace('agg_id', 'trade_id')
        fullnpath = make_get_filepath(npath + f + '/' + e)
        print(fullnpath)
        with open(fullnpath, 'w') as q:
            q.write(updated)


In [ ]:
updated

In [ ]:
any([[], []])

In [ ]:
await bot.create_orders
await bot.update_position()
bot.position

In [ ]:
a = '''Help on bybit in module ccxt.async_support.bybit object:

class bybit(ccxt.async_support.base.exchange.Exchange)
 |  bybit(config={})
 |  
 |  Base exchange class
 |  
 |  Method resolution order:
 |      bybit
 |      ccxt.async_support.base.exchange.Exchange
 |      ccxt.base.exchange.Exchange
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  amountToPrecision = amount_to_precision(self, symbol, amount)
 |  
 |  buildOhlcvc = build_ohlcvc(self, trades, timeframe='1m', since=None, limit=None)
 |  
 |  calculateFee = calculate_fee(self, symbol, type, side, amount, price, takerOrMaker='taker', params={})
 |  
 |  async cancelAllOrders = cancel_all_orders(self, symbol=None, params={})
 |  
 |  async cancelOrder = cancel_order(self, id, symbol=None, params={})
 |  
 |  cancelUnifiedOrder = cancel_unified_order(self, order, params={})
 |  
 |  async cancel_all_orders(self, symbol=None, params={})
 |  
 |  async cancel_order(self, id, symbol=None, params={})
 |  
 |  checkAddress = check_address(self, address)
 |  
 |  checkRequiredCredentials = check_required_credentials(self, error=True)
 |  
 |  checkRequiredDependencies = check_required_dependencies(self)
 |  
 |  commonCurrencyCode = common_currency_code(self, currency)
 |  
 |  convertOHLCVToTradingView = convert_ohlcv_to_trading_view(self, ohlcvs, t='t', o='o', h='h', l='l', c='c', v='v', ms=False)
 |  
 |  convertTradingViewToOHLCV = convert_trading_view_to_ohlcv(self, ohlcvs, t='t', o='o', h='h', l='l', c='c', v='v', ms=False)
 |  
 |  costToPrecision = cost_to_precision(self, symbol, cost)
 |  
 |  createLimitBuyOrder = create_limit_buy_order(self, symbol, *args)
 |  
 |  createLimitOrder = create_limit_order(self, symbol, *args)
 |  
 |  createLimitSellOrder = create_limit_sell_order(self, symbol, *args)
 |  
 |  createMarketBuyOrder = create_market_buy_order(self, symbol, amount, params={})
 |  
 |  createMarketOrder = create_market_order(self, symbol, *args)
 |  
 |  createMarketSellOrder = create_market_sell_order(self, symbol, amount, params={})
 |  
 |  async createOrder = create_order(self, symbol, type, side, amount, price=None, params={})
 |  
 |  async create_order(self, symbol, type, side, amount, price=None, params={})
 |  
 |  currencyId = currency_id(self, commonCode)
 |  
 |  currencyToPrecision = currency_to_precision(self, currency, fee)
 |  
 |  defineRestApi = define_rest_api(api, method_name, paths=[]) from builtins.type
 |  
 |  describe(self)
 |  
 |  editLimitBuyOrder = edit_limit_buy_order(self, id, symbol, *args)
 |  
 |  editLimitOrder = edit_limit_order(self, id, symbol, *args)
 |  
 |  editLimitSellOrder = edit_limit_sell_order(self, id, symbol, *args)
 |  
 |  async editOrder = edit_order(self, id, symbol, type, side, amount=None, price=None, params={})
 |  
 |  async edit_order(self, id, symbol, type, side, amount=None, price=None, params={})
 |  
 |  feeToPrecision = fee_to_precision(self, symbol, fee)
 |  
 |  async fetchBalance = fetch_balance(self, params={})
 |  
 |  fetchBidsAsks = fetch_bids_asks(self, symbols=None, params={})
 |  
 |  async fetchClosedOrders = fetch_closed_orders(self, symbol=None, since=None, limit=None, params={})
 |  
 |  async fetchCurrencies = fetch_currencies(self, params={})
 |  
 |  async fetchDepositAddress = fetch_deposit_address(self, code=None, since=None, limit=None, params={})
 |  
 |  async fetchDeposits = fetch_deposits(self, code=None, since=None, limit=None, params={})
 |  
 |  async fetchFees = fetch_fees(self)
 |  
 |  fetchFreeBalance = fetch_free_balance(self, params={})
 |  
 |  async fetchFullTickers = fetch_full_tickers(self, symbols=None, params={})
 |  
 |  fetchFundingFee = fetch_funding_fee(self, code, params={})
 |  
 |  fetchFundingFees = fetch_funding_fees(self, params={})
 |  
 |  async fetchL2OrderBook = fetch_l2_order_book(self, symbol, limit=None, params={})
 |  
 |  async fetchLedger = fetch_ledger(self, code=None, since=None, limit=None, params={})
 |  
 |  async fetchMarkets = fetch_markets(self, params={})
 |  
 |  async fetchMyTrades = fetch_my_trades(self, symbol=None, since=None, limit=None, params={})
 |  
 |  async fetchOHLCV = fetch_ohlcv(self, symbol, timeframe='1m', since=None, limit=None, params={})
 |  
 |  async fetchOhlcvc = fetch_ohlcvc(self, symbol, timeframe='1m', since=None, limit=None, params={})
 |  
 |  async fetchOpenOrders = fetch_open_orders(self, symbol=None, since=None, limit=None, params={})
 |  
 |  async fetchOrder = fetch_order(self, id, symbol=None, params={})
 |  
 |  async fetchOrderBook = fetch_order_book(self, symbol, limit=None, params={})
 |  
 |  async fetchOrderStatus = fetch_order_status(self, id, symbol=None, params={})
 |  
 |  async fetchOrderTrades = fetch_order_trades(self, id, symbol=None, since=None, limit=None, params={})
 |  
 |  async fetchOrders = fetch_orders(self, symbol=None, since=None, limit=None, params={})
 |  
 |  async fetchPartialBalance = fetch_partial_balance(self, part, params={})
 |  
 |  async fetchStatus = fetch_status(self, params={})
 |  
 |  async fetchTicker = fetch_ticker(self, symbol, params={})
 |  
 |  async fetchTickers = fetch_tickers(self, symbols=None, params={})
 |  
 |  async fetchTime = fetch_time(self, params={})
 |  
 |  fetchTotalBalance = fetch_total_balance(self, params={})
 |  
 |  async fetchTrades = fetch_trades(self, symbol, since=None, limit=None, params={})
 |  
 |  async fetchTradingFee = fetch_trading_fee(self, symbol, params={})
 |  
 |  async fetchTradingFees = fetch_trading_fees(self, params={})
 |  
 |  async fetchTransactions = fetch_transactions(self, code=None, since=None, limit=None, params={})
 |  
 |  fetchUnifiedOrder = fetch_unified_order(self, order, params={})
 |  
 |  fetchUsedBalance = fetch_used_balance(self, params={})
 |  
 |  async fetchWithdrawals = fetch_withdrawals(self, code=None, since=None, limit=None, params={})
 |  
 |  async fetch_balance(self, params={})
 |  
 |  async fetch_closed_orders(self, symbol=None, since=None, limit=None, params={})
 |  
 |  async fetch_deposits(self, code=None, since=None, limit=None, params={})
 |  
 |  async fetch_ledger(self, code=None, since=None, limit=None, params={})
 |  
 |  async fetch_markets(self, params={})
 |  
 |  async fetch_my_trades(self, symbol=None, since=None, limit=None, params={})
 |  
 |  async fetch_ohlcv(self, symbol, timeframe='1m', since=None, limit=None, params={})
 |  
 |  async fetch_open_orders(self, symbol=None, since=None, limit=None, params={})
 |  
 |  async fetch_order(self, id, symbol=None, params={})
 |  
 |  async fetch_order_book(self, symbol, limit=None, params={})
 |  
 |  async fetch_order_trades(self, id, symbol=None, since=None, limit=None, params={})
 |  
 |  async fetch_orders(self, symbol=None, since=None, limit=None, params={})
 |  
 |  async fetch_ticker(self, symbol, params={})
 |  
 |  async fetch_tickers(self, symbols=None, params={})
 |  
 |  async fetch_time(self, params={})
 |  
 |  async fetch_trades(self, symbol, since=None, limit=None, params={})
 |  
 |  async fetch_withdrawals(self, code=None, since=None, limit=None, params={})
 |  
 |  filterByArray = filter_by_array(self, objects, key, values=None, indexed=True)
 |  
 |  filterByCurrencySinceLimit = filter_by_currency_since_limit(self, array, code=None, since=None, limit=None)
 |  
 |  filterBySinceLimit = filter_by_since_limit(self, array, since=None, limit=None, key='timestamp', tail=False)
 |  
 |  filterBySymbol = filter_by_symbol(self, array, symbol=None)
 |  
 |  filterBySymbolSinceLimit = filter_by_symbol_since_limit(self, array, symbol=None, since=None, limit=None)
 |  
 |  filterByValueSinceLimit = filter_by_value_since_limit(self, array, field, value=None, since=None, limit=None, key='timestamp', tail=False)
 |  
 |  findBroadlyMatchedKey = find_broadly_matched_key(self, broad, string)
 |  
 |  handleErrors = handle_errors(self, httpCode, reason, url, method, headers, body, response, requestHeaders, requestBody)
 |  
 |  handleHttpStatusCode = handle_http_status_code(self, http_status_code, http_status_text, url, method, body)
 |  
 |  handle_errors(self, httpCode, reason, url, method, headers, body, response, requestHeaders, requestBody)
 |  
 |  initRestRateLimiter = init_rest_rate_limiter(self)
 |  
 |  isTextResponse = is_text_response(self, headers)
 |  
 |  async loadAccounts = load_accounts(self, reload=False, params={})
 |  
 |  async loadFees = load_fees(self, reload=False)
 |  
 |  async loadMarkets = load_markets(self, reload=False, params={})
 |  
 |  async loadMarketsHelper = load_markets_helper(self, reload=False, params={})
 |  
 |  async loadTimeDifference = load_time_difference(self, params={})
 |  
 |  async loadTradingLimits = load_trading_limits(self, symbols=None, reload=False, params={})
 |  
 |  async load_time_difference(self, params={})
 |  
 |  marketId = market_id(self, symbol)
 |  
 |  marketIds = market_ids(self, symbols)
 |  
 |  nonce(self)
 |  
 |  openapiGetApiKey = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiGetFundingPredictedFunding = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiGetFundingPrevFunding = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiGetFundingPrevFundingRate = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiGetOrderList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiGetStopOrderList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiGetWalletFundRecords = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiGetWalletRiskLimitList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiGetWalletWithdrawList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiPostOrderReplace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiPostPositionTradingStop = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiPostStopOrderCancel = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiPostStopOrderCreate = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiPostStopOrderReplace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapiPostWalletRiskLimit = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_get_api_key = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_get_funding_predicted_funding = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_get_funding_prev_funding = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_get_funding_prev_funding_rate = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_get_order_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_get_stop_order_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_get_wallet_fund_records = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_get_wallet_risk_limit_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_get_wallet_withdraw_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_post_order_replace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_post_position_trading_stop = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_post_stop_order_cancel = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_post_stop_order_create = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_post_stop_order_replace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  openapi_post_wallet_risk_limit = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  parseBalance = parse_balance(self, balance)
 |  
 |  parseBidAsk = parse_bid_ask(self, bidask, price_key=0, amount_key=0)
 |  
 |  parseBidsAsks = parse_bids_asks(self, bidasks, price_key=0, amount_key=1)
 |  
 |  parseJson = parse_json(self, http_response)
 |  
 |  parseLedger = parse_ledger(self, data, currency=None, since=None, limit=None, params={})
 |  
 |  parseLedgerEntry = parse_ledger_entry(self, item, currency=None)
 |  
 |  parseLedgerEntryType = parse_ledger_entry_type(self, type)
 |  
 |  parseOHLCV = parse_ohlcv(self, ohlcv, market=None)
 |  
 |  parseOhlcvs = parse_ohlcvs(self, ohlcvs, market=None, timeframe='1m', since=None, limit=None)
 |  
 |  parseOrder = parse_order(self, order, market=None)
 |  
 |  parseOrderBook = parse_order_book(self, orderbook, timestamp=None, bidsKey='Buy', asksKey='Sell', priceKey='price', amountKey='size')
 |  
 |  parseOrderStatus = parse_order_status(self, status)
 |  
 |  parseOrders = parse_orders(self, orders, market=None, since=None, limit=None, params={})
 |  
 |  parseTicker = parse_ticker(self, ticker, market=None)
 |  
 |  parseTimeInForce = parse_time_in_force(self, timeInForce)
 |  
 |  parseTrade = parse_trade(self, trade, market=None)
 |  
 |  parseTrades = parse_trades(self, trades, market=None, since=None, limit=None, params={})
 |  
 |  parseTradingViewOHLCV = parse_trading_view_ohlcv(self, ohlcvs, market=None, timeframe='1m', since=None, limit=None)
 |  
 |  parseTransaction = parse_transaction(self, transaction, currency=None)
 |  
 |  parseTransactionStatus = parse_transaction_status(self, status)
 |  
 |  parseTransactions = parse_transactions(self, transactions, currency=None, since=None, limit=None, params={})
 |  
 |  parse_ledger_entry(self, item, currency=None)
 |  
 |  parse_ledger_entry_type(self, type)
 |  
 |  parse_ohlcv(self, ohlcv, market=None)
 |  
 |  parse_order(self, order, market=None)
 |  
 |  parse_order_book(self, orderbook, timestamp=None, bidsKey='Buy', asksKey='Sell', priceKey='price', amountKey='size')
 |  
 |  parse_order_status(self, status)
 |  
 |  parse_ticker(self, ticker, market=None)
 |  
 |  parse_time_in_force(self, timeInForce)
 |  
 |  parse_trade(self, trade, market=None)
 |  
 |  parse_transaction(self, transaction, currency=None)
 |  
 |  parse_transaction_status(self, status)
 |  
 |  async performOrderBookRequest = perform_order_book_request(self, market, limit=None, params={})
 |  
 |  positionPostChangePositionMargin = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  position_post_change_position_margin = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  precisionFromString = precision_from_string(self, string)
 |  
 |  prepareRequestHeaders = prepare_request_headers(self, headers=None)
 |  
 |  priceToPrecision = price_to_precision(self, symbol, price)
 |  
 |  privateGetAccountLcp = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateGetExchangeOrderList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateGetExecutionList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateGetOrder = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateGetOrderList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateGetPositionList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateGetStopOrder = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateGetStopOrderList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateGetTradeClosedPnlList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateGetWalletBalance = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearGetFundingPredictedFunding = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearGetFundingPrevFunding = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearGetOrderList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearGetOrderSearch = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearGetPositionList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearGetStopOrderList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearGetStopOrderSearch = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearGetTradeClosedPnlList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearGetTradeExecutionList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostOrderCancel = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostOrderCancelAll = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostOrderCreate = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostOrderReplace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostPositionAddMargin = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostPositionSetAutoAddMargin = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostPositionSetLeverage = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostPositionSwitchIsolated = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostPositionTradingStop = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostStopOrderCancel = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostStopOrderCancelAll = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostStopOrderCreate = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostStopOrderReplace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinearPostTpslSwitchMode = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_get_funding_predicted_funding = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_get_funding_prev_funding = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_get_order_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_get_order_search = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_get_position_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_get_stop_order_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_get_stop_order_search = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_get_trade_closed_pnl_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_get_trade_execution_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_order_cancel = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_order_cancel_all = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_order_create = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_order_replace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_position_add_margin = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_position_set_auto_add_margin = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_position_set_leverage = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_position_switch_isolated = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_position_trading_stop = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_stop_order_cancel = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_stop_order_cancel_all = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_stop_order_create = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_stop_order_replace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privateLinear_post_tpsl_switch_mode = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privatePostOrderCancel = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privatePostOrderCancelAll = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privatePostOrderCancelall = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privatePostOrderCreate = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privatePostOrderReplace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privatePostStopOrderCancel = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privatePostStopOrderCancelAll = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privatePostStopOrderCancelall = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privatePostStopOrderCreate = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  privatePostStopOrderReplace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_get_account_lcp = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_get_exchange_order_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_get_execution_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_get_order = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_get_order_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_get_position_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_get_stop_order = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_get_stop_order_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_get_trade_closed_pnl_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_get_wallet_balance = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_post_order_cancel = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_post_order_cancelall = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_post_order_create = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_post_order_replace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_post_stop_order_cancel = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_post_stop_order_cancelall = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_post_stop_order_create = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  private_post_stop_order_replace = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetAccountRatio = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetAnnouncement = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetBigDeal = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetKlineList = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetLiqRecords = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetMarkPriceKline = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetOpenInterest = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetOrderBookL2 = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetOrderbookL2 = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetSymbols = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetTickers = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetTime = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicGetTradingRecords = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicLinearGetFundingPrevFundingRate = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicLinearGetKline = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicLinearGetMarkPriceKline = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicLinearGetRecentTradingRecords = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicLinearGetRiskLimit = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicLinear_get_funding_prev_funding_rate = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicLinear_get_kline = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicLinear_get_mark_price_kline = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicLinear_get_recent_trading_records = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  publicLinear_get_risk_limit = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_account_ratio = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_announcement = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_big_deal = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_kline_list = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_liq_records = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_mark_price_kline = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_open_interest = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_orderbook_l2 = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_symbols = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_tickers = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_time = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  public_get_trading_records = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  safeCurrency = safe_currency(self, currency_id, currency=None)
 |  
 |  safeCurrencyCode = safe_currency_code(self, currency_id, currency=None)
 |  
 |  safeMarket = safe_market(self, marketId, market=None, delimiter=None)
 |  
 |  safeSymbol = safe_symbol(self, marketId, market=None, delimiter=None)
 |  
 |  setHeaders = set_headers(self, headers)
 |  
 |  setMarkets = set_markets(self, markets, currencies=None)
 |  
 |  setSandboxMode = set_sandbox_mode(self, enabled)
 |  
 |  sign(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |  
 |  signMessageString = sign_message_string(self, message, privateKey)
 |  
 |  throwBroadlyMatchedException = throw_broadly_matched_exception(self, broad, string, message)
 |  
 |  throwExactlyMatchedException = throw_exactly_matched_exception(self, exact, string, message)
 |  
 |  userGetLeverage = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  userPostLeverageSave = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  user_get_leverage = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  user_post_leverage_save = request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from ccxt.async_support.base.exchange.Exchange:
 |  
 |  async __aenter__(self)
 |  
 |  async __aexit__(self, exc_type, exc, tb)
 |  
 |  __del__(self)
 |  
 |  __init__(self, config={})
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  async close(self)
 |  
 |  async fetch(self, url, method='GET', headers=None, body=None)
 |      Perform a HTTP request and return decoded JSON data
 |  
 |  async fetch2(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      A better wrapper over request for deferred signing
 |  
 |  async fetchOHLCVC(self, symbol, timeframe='1m', since=None, limit=None, params={})
 |  
 |  async fetch_currencies(self, params={})
 |  
 |  async fetch_deposit_address(self, code=None, since=None, limit=None, params={})
 |  
 |  async fetch_fees(self)
 |  
 |  async fetch_full_tickers(self, symbols=None, params={})
 |  
 |  async fetch_l2_order_book(self, symbol, limit=None, params={})
 |  
 |  async fetch_ohlcvc(self, symbol, timeframe='1m', since=None, limit=None, params={})
 |  
 |  async fetch_order_status(self, id, symbol=None, params={})
 |  
 |  async fetch_partial_balance(self, part, params={})
 |  
 |  async fetch_status(self, params={})
 |  
 |  async fetch_trading_fee(self, symbol, params={})
 |  
 |  async fetch_trading_fees(self, params={})
 |  
 |  async fetch_transactions(self, code=None, since=None, limit=None, params={})
 |  
 |  init_rest_rate_limiter(self)
 |  
 |  async load_accounts(self, reload=False, params={})
 |  
 |  async load_fees(self, reload=False)
 |  
 |  async load_markets(self, reload=False, params={})
 |  
 |  async load_markets_helper(self, reload=False, params={})
 |  
 |  async load_trading_limits(self, symbols=None, reload=False, params={})
 |  
 |  open(self)
 |  
 |  async perform_order_book_request(self, market, limit=None, params={})
 |  
 |  async sleep(self, milliseconds)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from ccxt.base.exchange.Exchange:
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  account(self)
 |  
 |  amount_to_precision(self, symbol, amount)
 |  
 |  build_ohlcvc(self, trades, timeframe='1m', since=None, limit=None)
 |  
 |  calculate_fee(self, symbol, type, side, amount, price, takerOrMaker='taker', params={})
 |  
 |  cancel_unified_order(self, order, params={})
 |  
 |  check_address(self, address)
 |      Checks an address is not the same character repeated or an empty sequence
 |  
 |  check_required_credentials(self, error=True)
 |  
 |  check_required_dependencies(self)
 |  
 |  common_currency_code(self, currency)
 |  
 |  convert_ohlcv_to_trading_view(self, ohlcvs, t='t', o='o', h='h', l='l', c='c', v='v', ms=False)
 |  
 |  convert_trading_view_to_ohlcv(self, ohlcvs, t='t', o='o', h='h', l='l', c='c', v='v', ms=False)
 |  
 |  cost_to_precision(self, symbol, cost)
 |  
 |  create_limit_buy_order(self, symbol, *args)
 |  
 |  create_limit_order(self, symbol, *args)
 |  
 |  create_limit_sell_order(self, symbol, *args)
 |  
 |  create_market_buy_order(self, symbol, amount, params={})
 |  
 |  create_market_order(self, symbol, *args)
 |  
 |  create_market_sell_order(self, symbol, amount, params={})
 |  
 |  currency(self, code)
 |  
 |  currency_id(self, commonCode)
 |  
 |  currency_to_precision(self, currency, fee)
 |  
 |  edit_limit_buy_order(self, id, symbol, *args)
 |  
 |  edit_limit_order(self, id, symbol, *args)
 |  
 |  edit_limit_sell_order(self, id, symbol, *args)
 |  
 |  fee_to_precision(self, symbol, fee)
 |  
 |  fetch_bids_asks(self, symbols=None, params={})
 |  
 |  fetch_free_balance(self, params={})
 |  
 |  fetch_funding_fee(self, code, params={})
 |  
 |  fetch_funding_fees(self, params={})
 |  
 |  fetch_total_balance(self, params={})
 |  
 |  fetch_unified_order(self, order, params={})
 |  
 |  fetch_used_balance(self, params={})
 |  
 |  filter_by_array(self, objects, key, values=None, indexed=True)
 |  
 |  filter_by_currency_since_limit(self, array, code=None, since=None, limit=None)
 |  
 |  filter_by_since_limit(self, array, since=None, limit=None, key='timestamp', tail=False)
 |  
 |  filter_by_symbol(self, array, symbol=None)
 |  
 |  filter_by_symbol_since_limit(self, array, symbol=None, since=None, limit=None)
 |  
 |  filter_by_value_since_limit(self, array, field, value=None, since=None, limit=None, key='timestamp', tail=False)
 |  
 |  find_broadly_matched_key(self, broad, string)
 |      A helper method for matching error strings exactly vs broadly
 |  
 |  handle_http_status_code(self, http_status_code, http_status_text, url, method, body)
 |  
 |  hashMessage(self, message)
 |  
 |  is_text_response(self, headers)
 |  
 |  market(self, symbol)
 |  
 |  market_id(self, symbol)
 |  
 |  market_ids(self, symbols)
 |  
 |  oath(self)
 |  
 |  parse_balance(self, balance)
 |  
 |  parse_bid_ask(self, bidask, price_key=0, amount_key=0)
 |  
 |  parse_bids_asks(self, bidasks, price_key=0, amount_key=1)
 |  
 |  parse_json(self, http_response)
 |  
 |  parse_ledger(self, data, currency=None, since=None, limit=None, params={})
 |  
 |  parse_ohlcvs(self, ohlcvs, market=None, timeframe='1m', since=None, limit=None)
 |  
 |  parse_orders(self, orders, market=None, since=None, limit=None, params={})
 |  
 |  parse_trades(self, trades, market=None, since=None, limit=None, params={})
 |  
 |  parse_trading_view_ohlcv(self, ohlcvs, market=None, timeframe='1m', since=None, limit=None)
 |  
 |  parse_transactions(self, transactions, currency=None, since=None, limit=None, params={})
 |  
 |  precision_from_string(self, string)
 |  
 |  prepare_request_headers(self, headers=None)
 |  
 |  price_to_precision(self, symbol, price)
 |  
 |  print(self, *args)
 |  
 |  privateKeyToAddress(self, privateKey)
 |  
 |  request(self, path, api='public', method='GET', params={}, headers=None, body=None)
 |      Exchange.request is the entry point for all generated methods
 |  
 |  safe_currency(self, currency_id, currency=None)
 |  
 |  safe_currency_code(self, currency_id, currency=None)
 |  
 |  safe_market(self, marketId, market=None, delimiter=None)
 |  
 |  safe_symbol(self, marketId, market=None, delimiter=None)
 |  
 |  set_headers(self, headers)
 |  
 |  set_markets(self, markets, currencies=None)
 |  
 |  set_sandbox_mode(self, enabled)
 |  
 |  signMessage(self, message, privateKey)
 |  
 |  sign_message_string(self, message, privateKey)
 |  
 |  soliditySha3(self, array)
 |  
 |  solidityTypes(self, array)
 |  
 |  solidityValues(self, array)
 |  
 |  throttle(self)
 |  
 |  throw_broadly_matched_exception(self, broad, string, message)
 |  
 |  throw_exactly_matched_exception(self, exact, string, message)
 |  
 |  vwap(self, baseVolume, quoteVolume)
 |  
 |  ----------------------------------------------------------------------
 |  Class methods inherited from ccxt.base.exchange.Exchange:
 |  
 |  define_rest_api(api, method_name, paths=[]) from builtins.type
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from ccxt.base.exchange.Exchange:
 |  
 |  aggregate(bidasks)
 |  
 |  array_concat(a, b)
 |  
 |  base16_to_binary(s)
 |  
 |  base58_to_binary(s)
 |      encodes a base58 string to as a big endian integer
 |  
 |  base64_to_binary(s)
 |  
 |  base64_to_string(s)
 |  
 |  base64urlencode(s)
 |  
 |  binary_concat(*args)
 |  
 |  binary_concat_array(array)
 |  
 |  binary_to_base16(s)
 |  
 |  binary_to_base58(b)
 |  
 |  binary_to_base64(s)
 |  
 |  capitalize(string)
 |  
 |  decimal_to_bytes(n, endian='big')
 |      int.from_bytes and int.to_bytes don't work in python2
 |  
 |  decode(string)
 |  
 |  deep_extend(*args)
 |  
 |  dmy(timestamp, infix='-')
 |  
 |  ecdsa(request, secret, algorithm='p256', hash=None, fixed_length=False)
 |  
 |  eddsa(request, secret, curve='ed25519')
 |  
 |  encode(string)
 |  
 |  encode_uri_component(uri, safe="~()*!.'")
 |  
 |  extend(*args)
 |  
 |  extract_params(string)
 |  
 |  filterBy(array, key, value=None)
 |  
 |  filter_by(array, key, value=None)
 |  
 |  from_wei(amount, decimals=18)
 |  
 |  groupBy(array, key)
 |  
 |  group_by(array, key)
 |  
 |  gzip_deflate(response, text)
 |  
 |  has_web3()
 |  
 |  hash(request, algorithm='md5', digest='hex')
 |  
 |  hmac(request, secret, algorithm=<built-in function openssl_sha256>, digest='hex')
 |  
 |  implode_params(string, params)
 |  
 |  in_array(needle, haystack)
 |  
 |  index_by(array, key)
 |  
 |  integer_divide(a, b)
 |      # python supports arbitrarily big integers
 |  
 |  integer_modulo(a, b)
 |  
 |  integer_pow(a, b)
 |  
 |  is_empty(object)
 |  
 |  is_json_encoded_object(input)
 |  
 |  iso8601(timestamp=None)
 |  
 |  json(data, params=None)
 |  
 |  jwt(request, secret, alg='HS256')
 |  
 |  key_exists(dictionary, key)
 |  
 |  keysort(dictionary)
 |  
 |  microseconds()
 |  
 |  milliseconds()
 |  
 |  msec()
 |  
 |  number_to_be(n, size)
 |  
 |  number_to_le(n, size)
 |  
 |  omit(d, *args)
 |  
 |  ordered(array)
 |  
 |  parse8601(timestamp=None)
 |  
 |  parse_date(timestamp=None)
 |  
 |  parse_timeframe(timeframe)
 |  
 |  pluck(array, key)
 |  
 |  rawencode(params={})
 |  
 |  remove0x_prefix(value)
 |  
 |  rfc2616(self, timestamp=None)
 |  
 |  round_timeframe(timeframe, timestamp, direction=3)
 |  
 |  rsa(request, secret, alg='RS256')
 |  
 |  safe_either(method, dictionary, key1, key2, default_value=None)
 |      A helper-wrapper for the safe_value_2() family.
 |  
 |  safe_float(dictionary, key, default_value=None)
 |  
 |  safe_float_2(dictionary, key1, key2, default_value=None)
 |  
 |  safe_integer(dictionary, key, default_value=None)
 |  
 |  safe_integer_2(dictionary, key1, key2, default_value=None)
 |  
 |  safe_integer_product(dictionary, key, factor, default_value=None)
 |  
 |  safe_integer_product_2(dictionary, key1, key2, factor, default_value=None)
 |  
 |  safe_string(dictionary, key, default_value=None)
 |  
 |  safe_string_2(dictionary, key1, key2, default_value=None)
 |  
 |  safe_string_lower(dictionary, key, default_value=None)
 |  
 |  safe_string_lower_2(dictionary, key1, key2, default_value=None)
 |  
 |  safe_string_upper(dictionary, key, default_value=None)
 |  
 |  safe_string_upper_2(dictionary, key1, key2, default_value=None)
 |  
 |  safe_timestamp(dictionary, key, default_value=None)
 |  
 |  safe_timestamp_2(dictionary, key1, key2, default_value=None)
 |  
 |  safe_value(dictionary, key, default_value=None)
 |  
 |  safe_value_2(dictionary, key1, key2, default_value=None)
 |  
 |  sec()
 |  
 |  seconds()
 |  
 |  signHash(hash, privateKey)
 |  
 |  sort_by(array, key, descending=False)
 |  
 |  string_to_base64(s)
 |  
 |  strip(string)
 |  
 |  sum(*args)
 |  
 |  to_array(value)
 |  
 |  to_wei(amount, decimals=18)
 |  
 |  totp(key)
 |  
 |  truncate(num, precision=0)
 |      Deprecated, use decimal_to_precision instead
 |  
 |  truncate_to_string(num, precision=0)
 |      Deprecated, todo: remove references from subclasses
 |  
 |  unique(array)
 |  
 |  unjson(input)
 |  
 |  urlencode(params={}, doseq=False)
 |  
 |  urlencode_with_array_repeat(params={})
 |  
 |  usec()
 |  
 |  uuid()
 |  
 |  uuidv1()
 |  
 |  ymd(timestamp, infix='-')
 |  
 |  ymdhms(timestamp, infix=' ')
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from ccxt.base.exchange.Exchange:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes inherited from ccxt.base.exchange.Exchange:
 |  
 |  accounts = None
 |  
 |  aiohttp_proxy = None
 |  
 |  aiohttp_trust_env = False
 |  
 |  api = None
 |  
 |  apiKey = ''
 |  
 |  asyncio_loop = None
 |  
 |  balance = None
 |  
 |  base58_alphabet = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrs...
 |  
 |  base58_decoder = None
 |  
 |  base58_encoder = None
 |  
 |  base_currencies = None
 |  
 |  certified = False
 |  
 |  commonCurrencies = {'BCC': 'BCH', 'BCHABC': 'BCH', 'BCHSV': 'BSV', 'DR...
 |  
 |  currencies = None
 |  
 |  currencies_by_id = None
 |  
 |  enableLastHttpResponse = True
 |  
 |  enableLastJsonResponse = True
 |  
 |  enableLastResponseHeaders = True
 |  
 |  enableRateLimit = False
 |  
 |  exceptions = None
 |  
 |  fees = {'funding': {'deposit': {}, 'withdraw': {}}, 'trading': {'perce...
 |  
 |  has = {'CORS': False, 'cancelAllOrders': False, 'cancelOrder': True, '...
 |  
 |  headers = None
 |  
 |  hostname = None
 |  
 |  httpExceptions = {'400': <class 'ccxt.base.errors.ExchangeNotAvailable...
 |  
 |  id = None
 |  
 |  ids = None
 |  
 |  lastRestPollTimestamp = 0
 |  
 |  lastRestRequestTimestamp = 0
 |  
 |  last_http_response = None
 |  
 |  last_json_response = None
 |  
 |  last_response_headers = None
 |  
 |  limits = {'amount': {'max': None, 'min': None}, 'cost': {'max': None, ...
 |  
 |  loaded_fees = {'funding': {'deposit': {}, 'withdraw': {}}, 'trading': ...
 |  
 |  logger = None
 |  
 |  markets = None
 |  
 |  marketsById = None
 |  
 |  markets_by_id = None
 |  
 |  minFundingAddressLength = 1
 |  
 |  myTrades = None
 |  
 |  name = None
 |  
 |  ohlcvs = None
 |  
 |  options = None
 |  
 |  orderbooks = None
 |  
 |  orders = None
 |  
 |  origin = '*'
 |  
 |  paddingMode = 5
 |  
 |  parseJsonResponse = True
 |  
 |  password = ''
 |  
 |  precision = None
 |  
 |  precisionMode = 2
 |  
 |  privateKey = ''
 |  
 |  pro = False
 |  
 |  proxies = None
 |  
 |  proxy = ''
 |  
 |  quote_currencies = None
 |  
 |  rateLimit = 2000
 |  
 |  rateLimitMaxTokens = 16
 |  
 |  rateLimitTokens = 16
 |  
 |  rateLimitUpdateTime = 0
 |  
 |  requiredCredentials = {'apiKey': True, 'login': False, 'password': Fal...
 |  
 |  requiresEddsa = False
 |  
 |  requiresWeb3 = False
 |  
 |  restPollerLoopIsRunning = False
 |  
 |  restRequestQueue = None
 |  
 |  secret = ''
 |  
 |  session = None
 |  
 |  status = {'eta': None, 'status': 'ok', 'updated': None, 'url': None}
 |  
 |  substituteCommonCurrencyCodes = True
 |  
 |  symbols = None
 |  
 |  tickers = None
 |  
 |  timeframes = None
 |  
 |  timeout = 10000
 |  
 |  token = ''
 |  
 |  trades = None
 |  
 |  transactions = None
 |  
 |  twofa = None
 |  
 |  uid = ''
 |  
 |  urls = None
 |  
 |  userAgent = None
 |  
 |  userAgents = {'chrome': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) App...
 |  
 |  verbose = False
 |  
 |  verify = True
 |  
 |  version = None
 |  
 |  walletAddress = ''
 |  
 |  web3 = None
'''

In [ ]:
idx = a.find('private_get_w')
idx

In [ ]:
print(a[idx:])

In [ ]:
await bot.cc.private_get_wallet_balance()